#  Capstone Project - The Battle of Neighborhoods)
------------------------------------------------------------------------------------------------

###  Business Problem

Everyone has there own taste of restaurant environment, someone like romantic, someone like the simple settings while other may prefer a quite place to dine alone. Those preferences varied from customer to customer.

My client, the dominant hot pot restaurant chain in China is looking to expand operation in North America. To start with, expand the first location in New York City (NYC).

###   Problem Statement

Recommend to the client the choice of neighbourhood to expand the chained restaurant. Data science tools will be used to discuss the following two problems: 1. NYC population and demographic characteristics. 2. Direct competitors in the locations.

###   Data Source

Data source 1: Wikipedia: 
https://en.wikipedia.org/wiki/New_York_City; 
https://en.wikipedia.org/wiki/Demographics_of_New_York_City


Web scraping techniques will be used to get population density data and demographic information from Wikipedia. From preliminary findings, the population density ranked from Manhattan, Brooklyn, Bronx, Queens and Staten Island. While on the other hand, Queens has the highest asian population density, followed by Manhattan, Brooklyn, Staten Island and Bronx. In this senario, asian customers are our target customer.

Data source 2: Geocoding.

Geographical coordinates will be obtained using Python Geocoder which address/neighbourhood of interest will be represented in latitude and longtitude. 


Date Source 3: Foursquare API. 

These data source will be used to explore various neighbourhoods in NYC and other hot pot restaurants venues within the neighbourhood. 

Finally, data extracted from the above data sources will be merged into one dataframe which will be further analyzed.

### Methodology section:

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests
from bs4 import BeautifulSoup
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
#!conda install -c conda-forge folium

import os
import folium # map rendering library
# Matplotlib and associated plotting modules

!pip install geocoder
import geocoder

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib as mp
import re
import csv
%matplotlib inline
from pandas.io.json import json_normalize

Solving environment: ...working... done

# All requested packages already installed.



### Scraping Wikipedia for Population and Demographic Data in New York City

In [2]:
response_obj = requests.get('https://en.wikipedia.org/wiki/New_York_City').text
soup = BeautifulSoup(response_obj,'lxml')
Neighborhoods_NYC_Table = soup.find('table', {'class':'wikitable sortable'})
### preparation of the table

In [3]:
rows = Neighborhoods_NYC_Table.select("tbody > tr")[3:8]

boroughs = []
for row in rows:
    borough = {}
    tds = row.select('td')
    borough["borough"] = tds[0].text.strip()
    borough["county"] = tds[1].text.strip()
    borough["population"] = float(tds[2].text.strip().replace(",",""))
    borough["gdp_billions"] = float(tds[3].text.strip().replace(",",""))
    borough["gdp_per_capita"] = float(tds[4].text.strip().replace(",",""))
    borough["land_sqm"] = float(tds[5].text.strip().replace(",",""))
    borough["land_sqkm"] = float(tds[6].text.strip().replace(",",""))
    borough["persons_sqm"] = float(tds[7].text.strip().replace(",",""))
    borough["persons_sqkm"] = float(tds[8].text.strip().replace(",",""))
    
    boroughs.append(borough)

print(boroughs)

[{'borough': 'The Bronx', 'county': 'Bronx', 'population': 1432132.0, 'gdp_billions': 42.695, 'gdp_per_capita': 29200.0, 'land_sqm': 42.1, 'land_sqkm': 109.04, 'persons_sqm': 34653.0, 'persons_sqkm': 13231.0}, {'borough': 'Brooklyn', 'county': 'Kings', 'population': 2582830.0, 'gdp_billions': 91.559, 'gdp_per_capita': 34600.0, 'land_sqm': 70.82, 'land_sqkm': 183.42, 'persons_sqm': 37137.0, 'persons_sqkm': 14649.0}, {'borough': 'Manhattan', 'county': 'New York', 'population': 1628701.0, 'gdp_billions': 600.244, 'gdp_per_capita': 360900.0, 'land_sqm': 22.83, 'land_sqkm': 59.13, 'persons_sqm': 72033.0, 'persons_sqkm': 27826.0}, {'borough': 'Queens', 'county': 'Queens', 'population': 2278906.0, 'gdp_billions': 93.31, 'gdp_per_capita': 39600.0, 'land_sqm': 108.53, 'land_sqkm': 281.09, 'persons_sqm': 21460.0, 'persons_sqkm': 8354.0}, {'borough': 'Staten Island', 'county': 'Richmond', 'population': 476179.0, 'gdp_billions': 14.514, 'gdp_per_capita': 30300.0, 'land_sqm': 58.37, 'land_sqkm': 15

In [4]:
response_obj = requests.get('https://en.wikipedia.org/wiki/Demographics_of_New_York_City').text
soup = BeautifulSoup(response_obj,'lxml')
Population_Census_Table = soup.select_one('.wikitable:nth-of-type(5)') #use css selector to target correct table.

jurisdictions = []
rows = Population_Census_Table.select("tbody > tr")[3:8]
for row in rows:
    jurisdiction = {}
    tds = row.select('td')
    jurisdiction["jurisdiction"] = tds[0].text.strip()
    jurisdiction["population_census"] = tds[1].text.strip()
    jurisdiction["%_white"] = float(tds[2].text.strip().replace(",",""))
    jurisdiction["%_black_or_african_amercian"] = float(tds[3].text.strip().replace(",",""))
    jurisdiction["%_Asian"] = float(tds[4].text.strip().replace(",",""))
    jurisdiction["%_other"] = float(tds[5].text.strip().replace(",",""))
    jurisdiction["%_mixed_race"] = float(tds[6].text.strip().replace(",",""))
    jurisdiction["%_hispanic_latino_of_other_race"] = float(tds[7].text.strip().replace(",",""))
    jurisdiction["%_catholic"] = float(tds[10].text.strip().replace(",",""))
    jurisdiction["%_jewish"] = float(tds[12].text.strip().replace(",",""))
    jurisdictions.append(jurisdiction)

print(jurisdictions)

[{'jurisdiction': 'Queens', 'population_census': '2,229,379', '%_white': 44.1, '%_black_or_african_amercian': 20.0, '%_Asian': 17.6, '%_other': 12.3, '%_mixed_race': 6.1, '%_hispanic_latino_of_other_race': 25.0, '%_catholic': 37.0, '%_jewish': 5.0}, {'jurisdiction': 'Manhattan', 'population_census': '1,537,195', '%_white': 54.4, '%_black_or_african_amercian': 17.4, '%_Asian': 9.4, '%_other': 14.7, '%_mixed_race': 4.1, '%_hispanic_latino_of_other_race': 27.2, '%_catholic': 11.0, '%_jewish': 9.0}, {'jurisdiction': 'Bronx', 'population_census': '1,332,650', '%_white': 29.9, '%_black_or_african_amercian': 35.6, '%_Asian': 3.0, '%_other': 25.7, '%_mixed_race': 5.8, '%_hispanic_latino_of_other_race': 48.4, '%_catholic': 14.0, '%_jewish': 5.0}, {'jurisdiction': 'Staten Island', 'population_census': '443,728', '%_white': 77.6, '%_black_or_african_amercian': 9.7, '%_Asian': 5.7, '%_other': 4.3, '%_mixed_race': 2.7, '%_hispanic_latino_of_other_race': 12.1, '%_catholic': 11.0, '%_jewish': 5.0}, {

In [157]:
df = pd.DataFrame(jurisdictions, columns=["jurisdiction","%_white", "%_black_or_african_amercian", "%_Asian", "%_other", "%_mixed_race", "%_hispanic_latino_of_other_race"])
df.head(10)

,jurisdiction,%_white,%_black_or_african_amercian,%_Asian,%_other,%_mixed_race,%_hispanic_latino_of_other_race
0,Queens,44.1,20.0,17.6,12.3,6.1,25.0
1,Manhattan,54.4,17.4,9.4,14.7,4.1,27.2
2,Bronx,29.9,35.6,3.0,25.7,5.8,48.4
3,Staten Island,77.6,9.7,5.7,4.3,2.7,12.1
4,NYC Total,44.7,26.6,9.8,14.0,4.9,27.0


### Foursquare: to find venues around New York City
Now that we have our location candidates, let's use Foursquare API to get info on restaurants in each neighborhood.

We're interested in venues in 'food' category, but only those that are proper restaurants - coffe shops, pizza places, bakeries etc. are not direct competitors so we don't care about those. So we will include in out list only venues that have 'restaurant' in category name, and we'll make sure to detect and include all the subcategories of specific 'Hot pot' category, as we need info on Asian restaurants in the neighborhoods.

In [6]:
# Get the NYC coordinate
address = 'New York City, United States'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of NYC are {}, {}.'.format(latitude, longitude))

C:\Users\derri\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of NYC are 40.7127281, -74.0060152.


In [7]:
neighborhood_latitude=40.7127281
neighborhood_longitude=-74.0060152

In [8]:
# @hidden_cell
CLIENT_ID = 'N0IGJ0J2U4UOF0M3D0RTALH0ALGSKWF5G3SBMPTBUDFLW1QD' #  Foursquare ID
CLIENT_SECRET = 'EHOHXWM3IKIHJ2NJ20HVFUTP1OMRMZ0GJ0NJYPXP3WMV2NM1' #  Foursquare Secret
VERSION = '20200503' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: N0IGJ0J2U4UOF0M3D0RTALH0ALGSKWF5G3SBMPTBUDFLW1QD
CLIENT_SECRET:EHOHXWM3IKIHJ2NJ20HVFUTP1OMRMZ0GJ0NJYPXP3WMV2NM1


In [9]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=N0IGJ0J2U4UOF0M3D0RTALH0ALGSKWF5G3SBMPTBUDFLW1QD&client_secret=EHOHXWM3IKIHJ2NJ20HVFUTP1OMRMZ0GJ0NJYPXP3WMV2NM1&v=20200503&ll=40.7127281,-74.0060152&radius=1000&limit=100'

In [10]:
# results display is hidden for report simplification 
results = requests.get(url).json()
#results

In [11]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [12]:
venues = results['response']['groups'][0]['items']
SGnearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues =SGnearby_venues.loc[:, filtered_columns]
# filter the category for each row
SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)
# clean columns
SGnearby_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]

SGnearby_venues.shape

C:\Users\derri\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


(100, 4)

In [13]:
SGnearby_venues.head(10)

,name,categories,lat,lng
0,The Bar Room at Temple Court,Hotel Bar,40.711448,-74.006802
1,"The Beekman, A Thompson Hotel",Hotel,40.711173,-74.006702
2,Alba Dry Cleaner & Tailor,Laundry Service,40.711434,-74.006272
3,City Hall Park,Park,40.712415,-74.006724
4,The Class by Taryn Toomey,Gym / Fitness Center,40.712753,-74.008734
5,The Wooly Daily,Coffee Shop,40.712137,-74.008395
6,Takahachi Bakery,Bakery,40.713653,-74.008804
7,Four Seasons Hotel New York Downtown,Hotel,40.712612,-74.009380
8,Pisillo Italian Panini,Sandwich Place,40.710530,-74.007526
9,Los Tacos No. 1,Taco Place,40.714267,-74.008756


### Data Obtained from NYC Open Data

In [116]:
filename = "Chinese_Restaurants.csv"

In [117]:
# Import Manhattan & Queens restaurant CSV From NYC OPEN DATA

NYC_data = pd.read_csv(filename)
NYC_data.head()


,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE
0,50094402,BOE SUN,Bronx,762,ALLERTON AVENUE,10467.0,7186549900,Chinese,07/17/2019,Violations were cited in the following area(s).,04L,Evidence of mice or live mice present in facil...,Y,12.0,A,07/17/2019,05/03/2020,Pre-permit (Operational) / Initial Inspection
1,50059930,HONG KONG RESTAURANT,Queens,19012,HILLSIDE AVE,11423.0,7187761818,Chinese,01/29/2018,Violations were cited in the following area(s).,04H,"Raw, cooked or prepared food is adulterated, c...",Y,19.0,NaN,NaN,05/03/2020,Cycle Inspection / Initial Inspection
2,50054760,OLD LUO YANG,Queens,13620,ROOSEVELT AVE,11354.0,3477324759,Chinese,01/23/2020,Violations were cited in the following area(s).,04A,Food Protection Certificate not held by superv...,Y,35.0,NaN,NaN,05/03/2020,Cycle Inspection / Initial Inspection
3,50075844,EAST OCEAN BUFFET,Brooklyn,1778,86 STREET,11214.0,2129206390,Chinese,05/08/2019,Violations were cited in the following area(s).,06A,Personal cleanliness inadequate. Outer garment...,Y,26.0,B,05/08/2019,05/03/2020,Cycle Inspection / Re-inspection
4,50071040,LET'S MAKAN,Manhattan,64A,BAYARD STREET,10013.0,6466494921,Chinese,03/11/2019,Violations were cited in the following area(s).,06D,"Food contact surface not properly washed, rins...",Y,15.0,NaN,NaN,05/03/2020,Cycle Inspection / Initial Inspection


In [118]:
df1= NYC_data[['DBA', 'BORO']]
df1.head()


,DBA,BORO
0,BOE SUN,Bronx
1,HONG KONG RESTAURANT,Queens
2,OLD LUO YANG,Queens
3,EAST OCEAN BUFFET,Brooklyn
4,LET'S MAKAN,Manhattan


In [119]:
Restaurant_count1 = df1['BORO'].value_counts().to_frame()
Restaurant_count1.rename(columns={'BORO': 'Amount_of_Restaurant'}, inplace=True)
Restaurant_count1

,Amount_of_Restaurant
Brooklyn,12567
Queens,11447
Manhattan,10800
Bronx,4214
Staten Island,1201


In [120]:
df1.shape

(40229, 2)

In [121]:
# Finding Hotpot Restaurant in NYC

df2 = df1[df1.DBA.str.contains('hot pot',case=False)]
df2.head()

,DBA,BORO
63,LAO JIE HOT POT,Brooklyn
115,LITTLE SHEEP MONGOLIAN HOT POT,Queens
648,LITTLE SHEEP MONGOLIAN HOT POT,Queens
668,CHICKEN HOT POT,Queens
806,LAO JIE HOT POT,Brooklyn


In [131]:
Restaurant_count2 = df2['BORO'].value_counts().to_frame()
Restaurant_count2.rename(columns={'BORO': 'Amount_of_Hotpot_Restaurant'}, inplace=True)

Restaurant_count2

,Amount_of_Hotpot_Restaurant
Queens,121
Manhattan,91
Brooklyn,69


From the above table, we can narrow down our borough selection to Manhattan, Queens and Brookyln. Only Queens, Manhattan and Brookyln have hot pot restaurant where Queens has the most and Brookyln has the least. Also, it is clearly indicate that, even though Brookyln has the biggest amount of restaurants but the amount of hotpot restaurant is much smaller than Manhattan and Queens. 

Now we can narrow down our choices to Manhattan and Queens.

Note: The below population data we also justify the assumption that Manhattan and Queens should be our final choices based on the asian population.

In [161]:
df.rename(columns={'jurisdiction': 'BORO'}, inplace=True)
df10 = df[['BORO','%_Asian']]
df10.head(2)

,BORO,%_Asian
0,Queens,17.6
1,Manhattan,9.4


### Results 
In order for a well-known, name-branded hotpot restaurant to successfully expand their footprint in North America, NYC is already pre-determined due to its diversity and asian population density

Our analysis shows that although there is a great number of restaurants in NYC (~40000 out of 5 boroughs), however, only three boroughs already have at least one hotpot restaurant, which are Manhattan, Queens and Brookyln. Although each one of those boroughs all have roughly 10,000 restaurant, in terms of hotpot, Queens has the highest count, 121, while brooklyn only has 69 in total. Thus brookyln is eliminated from the selection and our final choices become Queens and Manhattan. 

### Discussion

Throughout this project and searching for data, I notice that even though New York City is a highly diverse region, in terms of asian population (out target customers for the client), only Manhattan, Queens and Brookyln have a resonable amount of asian population and director hotpot restaurant compeition. This could also be a trememdous business opportunity to expand into non-asian customers or data is not well documented for boroughs of Staten Island and Bronx. 


### Conclusion 
In this study, I analyzed the population & demographic data in New York City as well as, restaurant and hotpot restaurant data.  I identified key factors which should be used when in location selection, which is the area where the target customers live in and popularity of the restaurant type in that area, where data is acquired from FourSquare and New York Open Data. I built both regression models and classification models to predict whether and how much a player would improve/decline. These models can be very useful in helping NBA team management in a number of ways. For example, it could help identify players to acquire, estimate the size of the contract to offer players, plan for performance changes of players already on the team, etc.
